We solve the Neoclassical Growth Model
\begin{align}
  \rho V(k) &= \max_{c} u(c) +  V'(k)(F(k)-\delta k - c) \\
  &u(c) = \frac{c^{1-\sigma}}{1-\sigma},~~~~ F(k)=k^\alpha
\end{align}

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sigma = 2
alpha = 0.3
delta = 0.05
rho = 0.05

In [3]:
kss = (alpha/(rho+delta))**(1/(1-alpha))
kmin = 0.001*kss
kmax = 2*kss

In [4]:
train_num = 1000
train = np.random.uniform(kmin,kmax,train_num)

In [5]:
train.shape

(1000,)

In [6]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='tanh', input_shape=[1]),
    layers.Dense(64, activation='tanh'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.0001)

  model.compile(loss='mse',
                optimizer=optimizer,
                )
  return model

In [7]:
model = build_model()

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________


In [9]:
initial_guess = -10*np.ones(train_num)
model.fit( train, initial_guess, epochs=200, validation_split = 0.2, verbose=0 )
value = model.predict([train])
value

array([[-9.995799 ],
       [-9.997003 ],
       [-9.997003 ],
       [-9.997115 ],
       [-9.997083 ],
       [-9.996374 ],
       [-9.996419 ],
       [-9.997113 ],
       [-9.9971   ],
       [-9.996572 ],
       [-9.997134 ],
       [-9.995915 ],
       [-9.997146 ],
       [-9.997022 ],
       [-9.996938 ],
       [-9.996815 ],
       [-9.997072 ],
       [-9.99702  ],
       [-9.997029 ],
       [-9.996947 ],
       [-9.997158 ],
       [-9.996698 ],
       [-9.995362 ],
       [-9.997117 ],
       [-9.99713  ],
       [-9.996912 ],
       [-9.997067 ],
       [-9.99466  ],
       [-9.995936 ],
       [-9.997159 ],
       [-9.996928 ],
       [-9.995836 ],
       [-9.996893 ],
       [-9.996715 ],
       [-9.997987 ],
       [-9.997039 ],
       [-9.997146 ],
       [-9.997113 ],
       [-9.997157 ],
       [-9.995313 ],
       [-9.99669  ],
       [-9.997155 ],
       [-9.996801 ],
       [-9.996158 ],
       [-9.997144 ],
       [-9.996684 ],
       [-9.997131 ],
       [-9.99

In [10]:
derivative = model.predict([train+1e-8])/(train.reshape((train_num, 1))+1e-8)
# derivative

In [11]:
update = 10
dt = 0.01

In [12]:
for update in range(update):
        value = model.predict([train])
        # the absolute vaule of this is WRONG 
        derivative = abs(model.predict([train+1e-8])/(train.reshape((train_num, 1))+1e-8))
        HJB = (derivative**(-1/sigma))**(1-sigma)/(1-sigma)+derivative*(train.reshape((train_num, 1))**alpha-delta*train.reshape((train_num, 1))-derivative**(-1/sigma))-rho*value
        value += dt*HJB
        model.fit( train, value, epochs=200, validation_split = 0.2, verbose=0 )
        print(update,  sum(abs(HJB)))

0 [2082.99160239]
1 [2017.20339555]
2 [1950.67014314]
3 [1886.01581263]
4 [1826.06715475]
5 [1763.23775044]
6 [1710.74386166]
7 [1663.5348705]
8 [1618.38181487]
9 [1575.86320421]
